In [1]:
import pandas as pd
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

deconfile='all_melanoma_CBC_SM_rawscore.txt'

CBCfile='Yale_samples_CBC_filtered_normalized.txt'

decondf=pd.read_csv(deconfile,sep='\t',index_col='Mixture')

decondf.head()

,Mono,Lymphocyte,PMN,Eosinophil
Mixture,,,,
YUALOE-20-4117,4188.341097,2373.541042,4348.262250,3118.363498
YUARGE-14-3173,2477.224681,1970.223042,1963.725767,1513.597369
YUBAIX-15-3422,4547.098180,2668.014794,7645.843258,3266.884824
YUBIFO-15-3368,3426.834433,2583.170097,3223.654493,2189.703070
YUCADA-14-3260,3440.280806,1852.850505,3598.950516,1869.630088


In [2]:
CBCdf=pd.read_csv(CBCfile,sep='\t',index_col='Mixture')
CBCdf.head()

,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %
Mixture,,,,,,
YUARGE-14-3173,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000
YUDUAR-20-4128,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172
YUNANCY-20-4157,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342
YUALOE-20-4117,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582
YUHONEY-20-4052,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871


In [3]:
merged=decondf.merge(CBCdf,left_index=True,right_index=True)
merged.head()

,Mono,Lymphocyte,PMN,Eosinophil,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %
Mixture,,,,,,,,,,
YUALOE-20-4117,4188.341097,2373.541042,4348.262250,3118.363498,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582
YUARGE-14-3173,2477.224681,1970.223042,1963.725767,1513.597369,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000
YUDUAR-20-4128,2908.788298,1421.675034,4451.145218,1829.257183,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172
YUHONEY-20-4052,3158.822098,1567.150455,4744.892563,2126.561818,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871
YUNANCY-20-4157,4046.154452,2396.668364,3629.194652,2800.993987,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342


In [4]:
#merged['EstLymphocytes']=merged['Naive CD4 T']+merged['CD4 TEM']+merged['CD4 TCM']+merged['Tregs']+merged['Naive CD8']+merged['CD8 TEM']+merged['CD8 TCM']+merged['CD8 TEMRA']+merged['NK']+merged['CD8TIL']
merged['EstLymphocytes']=merged['Lymphocyte']
merged.head()

,Mono,Lymphocyte,PMN,Eosinophil,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %,EstLymphocytes
Mixture,,,,,,,,,,,
YUALOE-20-4117,4188.341097,2373.541042,4348.262250,3118.363498,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582,2373.541042
YUARGE-14-3173,2477.224681,1970.223042,1963.725767,1513.597369,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000,1970.223042
YUDUAR-20-4128,2908.788298,1421.675034,4451.145218,1829.257183,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172,1421.675034
YUHONEY-20-4052,3158.822098,1567.150455,4744.892563,2126.561818,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871,1567.150455
YUNANCY-20-4157,4046.154452,2396.668364,3629.194652,2800.993987,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342,2396.668364


In [5]:
onlyrequireddf=merged[['EstLymphocytes','Mono','PMN','Eosinophil','Date','Sequencing batch','Neutrophils %','Lymphocytes %','Monocytes %','Eosinophils %']].copy()

onlyrequireddf

,EstLymphocytes,Mono,PMN,Eosinophil,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %
Mixture,,,,,,,,,,
YUALOE-20-4117,2373.541042,4188.341097,4348.262250,3118.363498,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582
YUARGE-14-3173,1970.223042,2477.224681,1963.725767,1513.597369,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000
YUDUAR-20-4128,1421.675034,2908.788298,4451.145218,1829.257183,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172
YUHONEY-20-4052,1567.150455,3158.822098,4744.892563,2126.561818,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871
YUNANCY-20-4157,2396.668364,4046.154452,3629.194652,2800.993987,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342
YUADD-20-4110,937.364361,2130.023279,3301.963503,1423.307424,10/21/20,EM-Seq-2,66.867470,20.481928,7.329317,5.321285
YUCEVO-20-4119,1186.395899,1831.041014,2485.974861,1679.735980,10/7/20,EM-Seq-2,62.139918,22.016461,15.226337,0.617284
YUFURL-17-3690,2075.017966,2563.871833,2877.380013,2160.449210,6/16/17,EM-Seq-2,55.991944,20.845921,10.574018,12.588117
YUHIMO-09-1242,1041.721216,1654.241200,1775.362714,1697.919268,9/8/09,EM-Seq-2,79.797980,13.131313,6.060606,1.010101


In [6]:
def plotter (xarray,yarray,celltype,outname):
    corr = pearsonr(xarray,yarray)

    corr_spear=spearmanr(xarray,yarray)
    text='r = %s, p = %s \nrho = %s, p = %s' % (format(corr[0], '.2f'), format(corr[1], ".2e"), format(corr_spear[0], ".2f"),format(corr_spear[1], ".2e")  )
    g=plt.scatter(x=xarray,y=yarray)
    #g.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), ncol=1)
    
    
    ax=sns.regplot(x=xarray,y=yarray,scatter=False,ci = None)
    anc = AnchoredText(text, loc="upper left", frameon=False,prop=dict(fontstyle="italic"))
    ax.add_artist(anc)
    
    savename=outname+celltype+".pdf"
    plt.xlabel('CBC (%)')
    plt.ylabel('LiquidTME (%)')
    plt.title(celltype)
    plt.savefig(savename,dpi=300,bbox_inches="tight")
    
    plt.close()

In [7]:

cohort='full'
currentdf=onlyrequireddf



plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+cohort)


cohort='BS'
currentdf=onlyrequireddf[onlyrequireddf['Sequencing batch']==cohort]

plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+cohort)


cohort='EM-Seq-1'
currentdf=onlyrequireddf[onlyrequireddf['Sequencing batch']==cohort]
plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+cohort)

cohort='EM-Seq-2'
currentdf=onlyrequireddf[onlyrequireddf['Sequencing batch']==cohort]
plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+cohort)

In [8]:
#####normalizing###

onlyrequireddf_normalized=onlyrequireddf.copy()
onlyrequireddf_normalized['EstOtherThan_lymphocytes']=100-onlyrequireddf_normalized['EstLymphocytes']

onlyrequireddf_normalized


,EstLymphocytes,Mono,PMN,Eosinophil,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %,EstOtherThan_lymphocytes
Mixture,,,,,,,,,,,
YUALOE-20-4117,2373.541042,4188.341097,4348.262250,3118.363498,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582,-2273.541042
YUARGE-14-3173,1970.223042,2477.224681,1963.725767,1513.597369,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000,-1870.223042
YUDUAR-20-4128,1421.675034,2908.788298,4451.145218,1829.257183,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172,-1321.675034
YUHONEY-20-4052,1567.150455,3158.822098,4744.892563,2126.561818,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871,-1467.150455
YUNANCY-20-4157,2396.668364,4046.154452,3629.194652,2800.993987,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342,-2296.668364
YUADD-20-4110,937.364361,2130.023279,3301.963503,1423.307424,10/21/20,EM-Seq-2,66.867470,20.481928,7.329317,5.321285,-837.364361
YUCEVO-20-4119,1186.395899,1831.041014,2485.974861,1679.735980,10/7/20,EM-Seq-2,62.139918,22.016461,15.226337,0.617284,-1086.395899
YUFURL-17-3690,2075.017966,2563.871833,2877.380013,2160.449210,6/16/17,EM-Seq-2,55.991944,20.845921,10.574018,12.588117,-1975.017966
YUHIMO-09-1242,1041.721216,1654.241200,1775.362714,1697.919268,9/8/09,EM-Seq-2,79.797980,13.131313,6.060606,1.010101,-941.721216


In [9]:
for index, row in onlyrequireddf_normalized.iterrows():

    corresum=onlyrequireddf_normalized.loc[index,['Mono','PMN','Eosinophil']].sum()

    onlyrequireddf_normalized.loc[index,['Mono','PMN','Eosinophil']]=(onlyrequireddf_normalized.loc[index,['Mono','PMN','Eosinophil']].divide(corresum,axis=0)*row['EstOtherThan_lymphocytes'])


In [10]:
onlyrequireddf_normalized['esttotal']=onlyrequireddf_normalized['EstLymphocytes']+onlyrequireddf_normalized['Mono']+onlyrequireddf_normalized['PMN']+onlyrequireddf_normalized['Eosinophil']

In [11]:
onlyrequireddf_normalized

,EstLymphocytes,Mono,PMN,Eosinophil,Date,Sequencing batch,Neutrophils %,Lymphocytes %,Monocytes %,Eosinophils %,EstOtherThan_lymphocytes,esttotal
Mixture,,,,,,,,,,,,
YUALOE-20-4117,2373.541042,-817.022091,-848.218001,-608.300949,10/6/20,EM-Seq-1,47.023209,36.528759,10.494450,5.953582,-2273.541042,100.0
YUARGE-14-3173,1970.223042,-778.054492,-616.773144,-475.395406,6/26/14,EM-Seq-1,59.000000,24.000000,12.000000,5.000000,-1870.223042,100.0
YUDUAR-20-4128,1421.675034,-418.369038,-640.205182,-263.100814,10/19/20,EM-Seq-1,63.434343,25.656566,9.191919,1.717172,-1321.675034,100.0
YUHONEY-20-4052,1567.150455,-462.047810,-694.045802,-311.056843,6/18/20,EM-Seq-1,67.137097,24.193548,7.560484,1.108871,-1467.150455,100.0
YUNANCY-20-4157,2396.668364,-887.015139,-795.607443,-614.045781,12/4/20,EM-Seq-1,45.390071,44.579534,8.004053,2.026342,-2296.668364,100.0
YUADD-20-4110,937.364361,-260.179290,-403.330109,-173.854962,10/21/20,EM-Seq-2,66.867470,20.481928,7.329317,5.321285,-837.364361,100.0
YUCEVO-20-4119,1186.395899,-331.718820,-450.369293,-304.307786,10/7/20,EM-Seq-2,62.139918,22.016461,15.226337,0.617284,-1086.395899,100.0
YUFURL-17-3690,2075.017966,-666.126292,-747.579677,-561.311998,6/16/17,EM-Seq-2,55.991944,20.845921,10.574018,12.588117,-1975.017966,100.0
YUHIMO-09-1242,1041.721216,-303.818038,-326.063223,-311.839955,9/8/09,EM-Seq-2,79.797980,13.131313,6.060606,1.010101,-941.721216,100.0


In [12]:
cohort='full'
currentdf=onlyrequireddf_normalized



plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+"normalized"+cohort)


cohort='BS'
currentdf=onlyrequireddf_normalized[onlyrequireddf_normalized['Sequencing batch']==cohort]

plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+"normalized"+cohort)


cohort='EM-Seq-1'
currentdf=onlyrequireddf_normalized[onlyrequireddf_normalized['Sequencing batch']==cohort]
plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+"normalized"+cohort)

cohort='EM-Seq-2'
currentdf=onlyrequireddf_normalized[onlyrequireddf_normalized['Sequencing batch']==cohort]
plotter(currentdf['Neutrophils %'],currentdf['PMN'],'Neutrophils',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Eosinophils %'],currentdf['Eosinophil'],'Eosinophil',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Monocytes %'],currentdf['Mono'],'Mono',outname=deconfile+"normalized"+cohort)
plotter(currentdf['Lymphocytes %'],currentdf['EstLymphocytes'],'Lymphocytes',outname=deconfile+"normalized"+cohort)